In [29]:
import pandas as pd
import numpy as np
from dash import Dash, html, dcc, callback, Output, Input
import plotly.graph_objects as go
import yfinance as yf

In [32]:
app = Dash(__name__)

app.layout = [
    html.H1(children='Stock Tracker', style={'textAlign':'center', 'color':'black'}),
    
    html.Div([
        html.Label('Input Stock Ticker: ', style={'fontSize': 20}),
        dcc.Input(
            id='ticker-input',
            type='text',
            placeholder='stock ticker, eg: APPL',
            style={'marginTop': 10, 'fontSize': 18, 'padding': '5px'}
        )
    ], style={'textAlign': 'center', 'marginTop': 50}),
    
    html.Div([
        html.Label('Select Value (1 to 5):', style={'fontSize': 20}),
        dcc.Slider(
            id='value-slider',
            min=1,
            max=5,
            step=1,
            marks={i: str(i) for i in range(1, 6)},
            value = 3
        )
    ], style={'textAlign': 'center', 'marginTop': 50}),

    dcc.Graph(id='candle-stick-graph'),
    dcc.Graph(id='volume-bar-chart')
]

@app.callback(
    [Output('candle-stick-graph', 'figure'),
     Output('volume-bar-chart', 'figure')],
    [Input('ticker-input', 'value'),
     Input('value-slider', 'value')]
)

def update_graph(ticker, days):
    if not ticker:  
        return go.Figure(), go.Figure()
        
    stock = yf.Ticker(ticker)
    df = stock.history(period=f"{days}d", interval='1m')
    df.to_csv('Data.csv')
    df = pd.read_csv('Data.csv')

    df['30_min_MA'] = df['Close'].rolling(window=30).mean()
    df['200_min_MA'] = df['Close'].rolling(window=200).mean()
    
    candlestick = go.Candlestick(x=df['Datetime'], low=df['Low'], high=df['High'], 
                             close=df['Close'], open=df['Open'])

    ma_30 = go.Scatter(
        x=df['Datetime'], y=df['30_min_MA'], 
        mode='lines', 
        name='30-min MA', 
        line=dict(color='rgba(0, 0, 255, 0.5)'))
    ma_200 = go.Scatter(
        x=df['Datetime'], y=df['200_min_MA'], 
        mode='lines', 
        name='200-min MA', 
        line=dict(color='rgba(255, 0, 0, 0.5)'))
    
    fig1 = go.Figure(data=[candlestick, ma_30, ma_200])
    fig1.update_layout(title=f'{ticker} Candlestick Chart',
                      xaxis_title='Date',
                      yaxis_title='Stock Price (USD)',
                      legend_title="Moving Averages"
                     )
    
    volume_bar = go.Bar(
        x=df['Datetime'], y=df['Volume'], 
        name='Volume', 
        marker_color='blue')

    max_volume = df['Volume'].max()
    tickvals = [10 ** i for i in range(0, int(np.log10(max_volume)) + 1)]
    ticktext = [f'{int(val):,}' for val in tickvals]
    fig2 = go.Figure(data=[volume_bar])
    fig2.update_layout(title=f'{ticker} Trading Volume',
                       xaxis_title='Date',
                       yaxis_title='Volume',
                       yaxis=dict(
                           type='log', 
                           tickvals=tickvals,
                           ticktext=ticktext,
                           autorange=True
                       ),
                      legend_title="Volume"
                      )
    
    return fig1, fig2


if __name__ == '__main__':
    app.run(debug=True)